In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data_train_125ft.csv")
df.drop(columns=["Unnamed: 0"],inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,0.0,87.0,1.0,14.0,712.000000,696.0,65.440519,240.0,0.703359,-0.246387,...,712.000000,696.0,65.707979,240.0,0.699256,-0.277091,783.821985,0.785714,9.0,6.0
1,2.0,49.0,1.0,13.0,712.000000,691.0,47.300458,158.0,0.884010,-0.332635,...,712.000000,691.0,47.483330,156.0,0.894198,-0.355280,380.188159,1.000000,13.0,12.0
2,1.0,86.0,1.0,9.0,1096.250000,1098.0,8.150920,28.0,-0.156675,-0.642487,...,1096.000000,1098.0,8.426150,28.0,-0.160466,-0.801230,808.359965,1.000000,8.0,6.0
3,1.0,70.0,1.0,8.0,1169.714286,1168.0,4.463000,14.0,0.642283,-0.722722,...,1169.428571,1168.0,4.237828,14.0,0.827427,-0.140248,1150.133430,1.000000,8.0,7.0
4,3.0,61.0,0.0,16.0,585.333333,586.0,2.890598,12.0,0.196283,0.113400,...,585.333333,586.0,3.155243,12.0,0.354663,-0.484056,198.042444,1.000000,16.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8343,0.0,72.0,1.0,16.0,619.466667,592.0,139.659522,480.0,0.480046,-0.842798,...,619.466667,594.0,139.562109,482.0,0.472791,-0.825913,468.155165,1.000000,16.0,8.0
8344,3.0,23.0,0.0,19.0,510.777778,512.0,7.091118,30.0,-0.059891,-0.269851,...,510.888889,512.0,6.870944,28.0,0.081059,-0.433829,309.210006,1.000000,19.0,18.0
8345,1.0,51.0,0.0,9.0,1090.250000,1088.0,9.769212,30.0,0.213875,-1.152589,...,1090.250000,1088.0,8.742854,28.0,0.436427,-0.891096,940.155678,1.000000,9.0,8.0
8346,3.0,45.0,0.0,26.0,370.320000,370.0,3.133305,10.0,0.358316,-0.885061,...,370.400000,370.0,3.098387,12.0,0.051640,-0.703333,856.813033,0.461538,1.0,6.0


In [3]:
x_train = df.iloc[:,1:].values
y_train = df.iloc[:,0].values

In [4]:
X = x_train
y = y_train

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X = sc.fit_transform(X)

In [6]:
from sklearn.metrics import confusion_matrix
def confusion_matrix_scorer(clf=None,X=None, y=None,y_pred=None,y_prob=None):
    if clf != None:
        y_pred = clf.predict(X)
        y_prob = clf.predict_proba(X)[:,1]
    cm = confusion_matrix(y,y_pred)
    acc_arr = []
    precision_arr = []
    recall_arr = []
    specificity_arr = []
    f1_arr = []
    for c in range(0,len(cm)):
        c = len(cm)-1
        tp = cm[c,c]
        fp = sum(cm[:,c]) - cm[c,c]
        fn = sum(cm[c,:]) - cm[c,c]
        tn = sum(np.delete(sum(cm)-cm[c,:],c))
        acc_arr.append((tp+tn) / (tp+fn+tn+fp))
        recall_arr.append(tp/(tp+fn))
        precision_arr.append(tp/(tp+fp))
        specificity_arr.append(tn/(tn+fp))
        f1_arr.append(2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn)))))
    return {'acc':np.nanmean(acc_arr),'precision':np.nanmean(precision_arr),'specificity':np.nanmean(specificity_arr),'recall':np.nanmean(recall_arr),'f1_score':np.nanmean(f1_arr)}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier()
params = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_depth': [3,4,5],
    'splitter':['best', 'random'],
    'max_features':['auto', 'sqrt', 'log2']
}
grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, verbose=5, return_train_score=True,refit=True)
grid_model = grid_search.fit(X,y)

In [8]:
grid_model.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'sqrt',
 'splitter': 'best'}